In [1]:
# Import liberaries and functions
import pandas as pd
import glob
import time
from bs4 import BeautifulSoup
import re
import requests
from random import random
from tqdm.auto import tqdm
from Data.data import load_data

In [ ]:
# Get firm category per CIK
# def make_request(url, params=None, headers=None, max_retries = 4):

#     current_tries = 1
#     while current_tries < max_retries:
#         response = requests.get(url=url, params=params, headers=headers)
        
#         if response.status_code == 200:
#             # time.sleep(random())
#             return response
#         else:
#             time.sleep(random()+0.1)
#             current_tries += 1
            
#     return requests.get(url=url, params=params, headers=headers)

# headers = {'User-Agent': 'kuleuven'}

# def get_category(CIK):
#     res = make_request(f"https://data.sec.gov/submissions/CIK{CIK:010}.json", headers=headers)
    
#     if res.status_code == 200:
#         temp = res.json()['category']
#     else:
#         temp = res.status_code

#     return temp

# CIKs = files["CIK"].astype(int).unique()
# cat_list = [get_category(cik) for cik in tqdm(CIKs)]
# cat_df = pd.DataFrame({"CIK": CIKs, "category": cat_list})
# cat_df.to_csv("category.csv")

In [11]:
# Get all raw html/text file paths
all_files = glob.glob('Data\RiskData\*')

all_files_df = pd.DataFrame(all_files, columns=["path"])
all_files_df["name"] = all_files_df["path"].apply(lambda x: x.split("\\")[-1])
all_files_df["extension"] = all_files_df["name"].apply(lambda x: x.split(".")[-1])
all_files_df["CIK"] = all_files_df["name"].apply(lambda x: x.split("-")[0]).astype(int)

all_files_df["report_dt"] = pd.to_datetime(
    all_files_df["name"].apply(lambda x: x.split("-")[2][1:])
)
all_files_df["filing_dt"] = pd.to_datetime(
    all_files_df["name"].apply(lambda x: x.split("-")[1][1:])
)

In [ ]:
# Filter files for the time period and drop duplicates
files = all_files_df.copy()[
    (all_files_df["report_dt"].dt.year > 2005) & 
    (all_files_df["report_dt"].dt.year < 2022)
    ]
    
files.sort_values(by=["CIK", "report_dt", "filing_dt"], inplace=True)
files.drop_duplicates(subset=["CIK", "report_dt"], keep='last', inplace=True)
files.loc[:, "CIK"] = files["CIK"].astype(int)

# # Save all files to train the W2V model
# files.to_csv("Data\\all_files.csv", index=False)

In [2]:
# Generate the ETM training data
RF_df = pd.read_csv('Data\W2V_train_3.csv', index_col=0).dropna()

c:\Users\u0147656\Anaconda3\envs\ETM_env\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
cat_df = pd.read_csv("Data/category.csv", index_col=0)

# Find smaller reporting companies
regex = r"(Smaller reporting company)"
cat_df["category"] = cat_df["category"].str.extract(regex)[0].str.replace(regex, 'SRC', regex=True)

# Firms industry info
all_co = pd.read_excel("Data/all_companies.xlsx")

In [4]:
# Add firm categories to the files df
RF_df_extend = pd.merge(RF_df, cat_df, on="CIK", how="left")

# Add firm industry to the Risk Factors df
RF_df_extend = pd.merge(RF_df_extend, all_co[["CIK", "SIC", "Industry"]], on="CIK", how='left')

In [ ]:
print(RF_df_extend[["CIK", "report_dt", "filing_dt"]].drop_duplicates().shape)
print(RF_df_extend["CIK"].nunique())

In [5]:
# Drop SRC firms and finance industry
RF_no_Fin = RF_df_extend.copy()[
    # (RF_df_extend["category"] != "SRC") &
    (RF_df_extend["Industry"] != "Office of Finance") & 
    (RF_df_extend["Industry"] != "Office of Structured Finance")
]
RF_no_Fin.reset_index(drop=True, inplace=True)

In [6]:
word_cnt = RF_no_Fin['cleaned_txt'].astype('str').map(lambda x: len(x.split()))
word_cnt.describe()

count    1.394277e+06
mean     1.158323e+02
std      1.081632e+02
min      1.000000e+00
25%      5.600000e+01
50%      9.400000e+01
75%      1.490000e+02
max      1.475800e+04
Name: cleaned_txt, dtype: float64

In [7]:
RF_no_Fin = RF_no_Fin[word_cnt >= word_cnt.quantile(0.1)]

In [8]:
rf_count = RF_no_Fin.groupby(['CIK', 'report_dt'])['cleaned_txt'].transform('count')
rf_count.describe()

count    1.255026e+06
mean     4.541601e+01
std      2.460240e+01
min      1.000000e+00
25%      2.800000e+01
50%      4.000000e+01
75%      5.800000e+01
max      2.400000e+02
Name: cleaned_txt, dtype: float64

In [ ]:
RF_no_Fin = RF_no_Fin[
    (rf_count>rf_count.quantile(0.005))&
    (rf_count<rf_count.quantile(0.999))
]

In [ ]:
print(RF_no_Fin[["CIK", "report_dt", "filing_dt"]].drop_duplicates().shape)
print(RF_no_Fin["CIK"].nunique())

In [ ]:
RF_no_Fin.rename(columns={'Unnamed: 0.1': 'rf_seq'}, inplace=True)

In [ ]:
RF_no_Fin.shape

In [ ]:
RF_no_Fin.to_csv("Data\clean_docs_3.csv", index=False)

In [ ]:
RF_no_Fin = pd.read_csv("Data\clean_docs_3.csv")
Item1A = pd.read_csv("Data\Item1As.csv", index_col=0)

In [ ]:
Item1A = Item1A[Item1A.CIK.isin(RF_no_Fin.CIK.unique())]
Item1A.to_csv("Data\Item1As.csv", index=False)

Code try

In [ ]:
get_CIKs = pd.read_csv('Data\get_CIKs.csv', index_col=0)
missing_CIKs = [CIK for CIK in get_CIKs["CIK"].unique() if CIK not in all_files_df["CIK"].unique()]
missing_CIKs_df = get_CIKs.set_index("CIK").loc[missing_CIKs].reset_index()
missing_CIKs_df.to_csv("missing_CIKs_df.csv")

In [ ]:
RF_no_SRC = pd.read_csv("Data\RF_df.csv", index_col=0).reset_index(drop=True)
RF_SRC = pd.read_csv("Data\RF_SRC.csv", index_col=0).reset_index(drop=True)
RF_df = pd.concat([RF_no_SRC, RF_SRC]).reset_index(drop=True)
RF_df.to_csv("Data\W2V_train.csv")

In [ ]:
cleaned_data = pd.read_csv("Data\clean_docs.csv")
word_cnt = cleaned_data['cleaned_txt'].astype('str').map(lambda x: len(x.split()))

In [ ]:
word_cnt.describe()

In [ ]:
print(int(word_cnt.quantile(q=0.999)))

In [ ]:
print(word_cnt.quantile(q=0.001))

In [ ]:
# from bs4 import BeautifulSoup
import re
import spacy
# from spacy.attrs import SENT_START
from time import strftime, gmtime
import pandas as pd
import glob
nlp = spacy.load('en_core_web_sm')
from file_preprocess import file_preprocess

In [ ]:
test_files = [
    'Data\\RiskData\\1377888-R20130415-C20121231-F13-0.htm',
    'Data\\RiskData\\20-R20090313-C20090103-F58-0.htm',
    'Data\\RiskData\\1750-R20190718-C20190531-F66-0.htm',
    'Data\\RiskData\\2488-R20180227-C20171230-F42-0.htm',
    'Data\\RiskData\\1505732-R20200228-C20191231-F40-0.htm',
    'Data\\RiskData\\39899-R20080228-C20071230-F03-0.htm',
    'Data\\RiskData\\3153-R20210218-C20201231-F06-0.htm',
    'Data\\RiskData\\3453-R20200228-C20191231-F57-0.htm',
    'Data\\RiskData\\6494-R20170324-C20161231-F58-0.htm',
    'Data\\RiskData\\1000234-R20070316-C20061231-F01-0.htm',
    'Data\\RiskData\\1004980-R20200218-C20191231-F09-0.htm',
    'Data\\RiskData\\100517-R20200225-C20191231-F10-0.htm',
    'Data\\RiskData\\1007019-R20070129-C20061031-F01-0.htm',
    'Data\\RiskData\\1003078-R20211020-C20210828-F99-0.htm',
    'Data\\RiskData\\1005409-R20140313-C20131231-F18-0.htm',
    'Data\\RiskData\\1005757-R20070301-C20061231-F01-0.htm',
    'Data\\RiskData\\1536089-R20200115-C20190930-F42-0.htm',
    'Data\\RiskData\\1423746-R20130424-C20121231-F30-0.htm',
    'Data\\RiskData\\20212-R20100302-C20091231-F63-0.htm',
    'Data\\RiskData\\1789029-R20210602-C20201231-F86-0.htm',
    'Data\\RiskData\\30140-R20070612-C20070331-F01-0.htm',
    'Data\\RiskData\\1727196-R20210309-C20201231-F41-0.htm'
]

In [ ]:
with open(test_files[-1], "rb") as f:
    html_text = f.read()
    
RFs = file_preprocess(html_text)

In [ ]:
RFs = []
for file in test_files:
    with open(file, "rb") as f:
        html_text = f.read()

        RFs.append(file_preprocess(html_text))

In [ ]:
RF_df = pd.read_csv('Data\RFs_all.csv').dropna()
RF_df.shape

In [ ]:
rf_count = RF_df.groupby(['CIK', 'report_dt'])['Item 1A'].count()
rf_count.describe()

In [ ]:
word_cnt = RF_df['Item 1A'].astype('str').map(lambda x: len(x.split()))
word_cnt.describe()

In [ ]:
pattern_1 = r"table\s+of\s+content[s]?"
pattern_2 = r"(item\W*)?1a"
pattern_3 = r"risk\s+factor[s]?"

pattern = r"(table\s+of\s+content[s]?)|((item\W*)?1a)|(risk\s+factor[s]?)"

In [ ]:
text = u"Our Series D Preferred Stock u.s. or us or US entitles the holder c to cumulative dividends, payable quarterly, at an annual rate of (i) 8% of the Stated Value of $1.00 during the three year period commencing on the date of issue, and (ii) 12% of the Stated table of content Value commencing three years after the date risk factor of issue item 1A."

In [ ]:
from Data.data import nlp_clean, bigram

In [ ]:
sample = RF_df["Item 1A"].sample(1000)

In [ ]:
for t in nlp(text):
    print(t, t.is_alpha, t.pos_, t.shape_.lower())

In [ ]:
list(nlp_clean(nlp(text)))

In [ ]:
sample.str.replace(pattern, " ", case=False, regex=True)

In [ ]:
# Replace some unwanted patterns in the RFs
pattern = r"(table\s+of\s+content[s]?)|((item\W*)?1a)|(risk\s+factor[s]?)"
RF_df["Item 1A"] = RF_df["Item 1A"].str.replace(pattern, " ", case=False, regex=True)

In [ ]:
sample = RF_df["Item 1A"][word_cnt<=2]
corpus = nlp.pipe(sample.to_list())
clean_corpus = [list(d) for d in (nlp_clean(doc, lemma=True) for doc in corpus)]

def token(text):
    return text

transformed_sents = bigram(
    raw_data = clean_corpus, 
    tokenizer=token,
    min_cnt = 0.00001
)

In [ ]:
cleaned_data = sample.to_frame()
cleaned_data.loc[:, 'cleaned_txt'] = [" ".join(d) for d in transformed_sents]

In [ ]:
RF_df["Item 1A"][word_cnt<=3]

In [5]:
df = pd.read_csv("Data/filings_df.csv")

In [6]:
df = df[(df["reportDate"]>="2006-01-01")&(df["reportDate"]<="2022-06-30")]

In [13]:
sample = pd.read_excel("Sample.xlsx")
data = agg_tops[[
    'CIK', 'filing_dt', 'report_dt', 'rprt_length'
]].copy()

data["reported"] = agg_tops['Topic'].map(len)
SP500 = data[data['CIK'].isin(sample['CIK'])]
SP500.to_excel("Sample_Olivia.xlsx", index=False)